In [7]:
import os
import shutil
import re
import subprocess
import math

from IPython.core.debugger import set_trace
from importlib import reload

In [30]:
# Given a complete filepath to a video, returns an integer of the number of frames in the video.
def video_data(filename):
    # Run FFmpeg to get information on the given video
    output = subprocess.run(['ffmpeg', '-i', filename, '-map', '0:v:0', '-c', 'copy', '-f', 'null', '-'], capture_output = True, text = True, check=True)
    # FFmpeg return a CompletedProcess data type. In order to process the information, change it to a string data type.
    output = str(output)
    # Use regular expression matching to extract video data
    frame_text = re.findall('frame= *\d+', output)[0] # Regular expression: "frame=" + (zero or more spaces or tabs) + (one or more digits)
    resolution_text = re.findall(', \d+x\d+', output)[0]
    # Get the number that comes after 'frame ='
    frame_num = ''.join(char for char in frame_text if char.isdigit())
    width = int(''.join(char for char in resolution_text.split('x')[0] if char.isdigit()))
    height = int(''.join(char for char in resolution_text.split('x')[1] if char.isdigit()))
    return (frame_num, width, height)

In [ ]:
#video_data(r'C:\Users\John\PythonVenvs\VideoSuperResolution\Scripts\Raw Data\Raw, Half-Size, and PNGs\2019 AERIAL _ Stock Footage\2019 AERIAL _ Stock Footage_half_size.mp4')

In [ ]:
for folder in os.listdir(os.getcwd()):
    os.chdir(folder)
    filename = list(filter(lambda x: x.endswith('_half_size.mp4'), os.listdir(os.getcwd())))[0] # Filename of the half-size video
    num_frames, width, height = video_data(os.getcwd() + '\\' + filename)

    new_width = width - (width % 256)
    new_height = height - (height % 256)
    cropped_filename = filename[:-4] + '_cropped.mp4'
    crop_command = 'crop=' + str(new_width) + ':' + str(new_height) + ':' \
    + str(math.floor((width % 265) / 2)) + ':' + str(math.floor((height % 265) / 2))
    subprocess.run(['ffmpeg', '-y', '-i', filename, '-filter:v', crop_command, cropped_filename])
    for x_pixel_index in list(range(0, new_width, 256)):
        for y_pixel_index in list(range(0, new_height, 256)):
            crop_command = 'crop=' + ':'.join(['256', '256', str(x_pixel_index), str(y_pixel_index)]) + ''
            patch_filename = cropped_filename[:-4] + str(int(x_pixel_index / 256)) + str(int(y_pixel_index / 256)) + '.mp4'
            subprocess.run(['ffmpeg', '-y', '-i', cropped_filename, '-filter:v', crop_command, patch_filename])
            if os.path.exists(patch_filename[:-4]):
                shutil.rmtree(patch_filename[:-4])
            os.mkdir(patch_filename[:-4])
            shutil.move(patch_filename, patch_filename[:-4] + '\\' + patch_filename)

    os.chdir('..')   

In [31]:
os.chdir(r'C:\Users\John\PythonVenvs\VideoSuperResolution\Scripts\Raw Data\Raw, Half-Size, and PNGs')
print(os.getcwd())

# For each file in the directory, split the full-length videos into individual .png images
# os.getcwd() returns a string of the current directory (the one set up above). os.listdir() then list every file in that directory.
for folder in [x for x in os.listdir(os.getcwd()) if not x.endswith('.bat')][4:]:
    os.chdir(folder)
    for patch_folder in (x for x in os.listdir(os.getcwd()) if not x.endswith('.mp4')):
        os.chdir(patch_folder)
        filename = os.listdir(os.getcwd())[0] # Filename of the patch of video
        num_frames, a, b = video_data(os.getcwd() + '\\' + filename)
        subprocess.run(['ffmpeg', '-i', filename, '-vf', 'select=\'between(n\\,0\\,' + num_frames + ')\'', '-vsync', '0', '-start_number', '0', 'frame%d.png'], check=True)
        os.chdir('..') # Back up to the directory with the list of patch folders
    
    os.chdir('..') # Back up to the main video directory
    

C:\Users\John\PythonVenvs\VideoSuperResolution\Scripts\Raw Data\Raw, Half-Size, and PNGs


CalledProcessError: Command '['ffmpeg', '-i', 'C:\\Users\\John\\PythonVenvs\\VideoSuperResolution\\Scripts\\Raw Data\\Raw, Half-Size, and PNGs\\Family Building Sand Castle Together Stock Footage\\Family Building Sand Castle Together Stock Footage_half_size_cropped00\\Family Building Sand Castle Together Stock Footage_half_size_cropped00.mp4', '-map', '0:v:0', '-c', 'copy', '-f', 'null', '-']' returned non-zero exit status 1.

In [ ]:
# Halve the raw video sizes to get the HR frames 
os.chdir(r'Raw Data\Just Raw Videos')

for folder in os.listdir(os.getcwd()):
    os.chdir(folder)
    filename = os.listdir(os.getcwd())[0] # Filename of the only file in the folder. Each folder should have exactly one video file
    # ffmpeg = subprocess.run(['ffmpeg', '-i', "\"" + filename + "\"", '-vf', '"scale=iw/2:ih/2"', "\"" + filename[:-4] + '_half_size.mp4' + "\""])
    ffmpeg = subprocess.run(['ffmpeg', '-y', '-i', filename, '-vf', 'scale=iw/2:ih/2', filename[:-4] + '_half_size.mp4'])
    os.chdir('..') # Back up to the directory with the list of video folders


In [8]:
reload(vid)

NameError: name 'vid' is not defined